In [ ]:
import numpy as np                                                                    # Importing required libraries
from scipy.stats import norm
import yfinance as yf
import pandas as pd
import random
from scipy.optimize import fsolve
from scipy.stats import norm
from sympy import nsolve, Symbol


##################### This part downloads the financial data and organizes the relevant information #####################


tickers = ['TSLA', 'SIE.DE', 'OMV.VI']                                                # Creating list of Tickers

start_date = '2014-01-01'                                                             # Start of time interval
end_date = '2023-12-31'                                                               # End of time interval


all_data = yf.download(tickers, start=start_date, end=end_date)                       # Downloading all financial data of the stocks, including Adj Close, Close, High, Low, Open and Volume.

data = pd.DataFrame(all_data['Adj Close'])                                            # Selecting adjusted close and saving it into a new dataframe



for i in range(len(tickers)):                                                         # Starting loop for downloading information about the number of outstanding shares

  stock_info = yf.Ticker(tickers[i]).info                                             # Downloading information, one by one, for all three tickers

  shares_outstanding = stock_info.get("sharesOutstanding", "N/A")                     # Extracting relevant information. Remark: This is the number of shares outstanding as of 21.01.2024. I couldn't get free historical daily data on this, so the calculation
                                                                                      # won't be completely exact. As soon as I find reliable daily data, I'll change this line, so that the results are more exact.

  total_debt = stock_info.get("totalDebt", "N/A")                                     # Extracting relevant information. Total debt has the same problem, as shares outstanding: it is only one number, the total debt as of 21.04.2024. This will also be
                                                                                      # changed as soon as reliable daily data is available but for the time being, we do the calculations with this number.

  column_name_1 = tickers[i] + '_shares_outstanding'                                  # Naming column so that it is called TICKER_shares_outstanding

  data[column_name_1] = shares_outstanding                                            # Creating the column that contains the number of shares outstanding

  column_name_2 = tickers[i] + '_market_cap'                                          # Naming column so that it is called TICKER_market_cap

  data[column_name_2] = shares_outstanding * data[tickers[i]]                         # Calculating the column for market capitlaization

  data = data.drop([column_name_1], axis = 1)                                         # Removing column with the number of shares outstanding

  column_name_3 = tickers[i] + '_total_debt'                                          # Naming column so that it is called TICKER_total_debt

  data[column_name_3] = total_debt                                                    # Creating the column that contains the total debt



data.rename(columns = {'OMV.VI':'OMV.VI_adj_close', 'SIE.DE':'SIE.DE_adj_close', 'TSLA':'TSLA_adj_close'}, inplace = True)                                                                                   # Renaming columns so that they are called TICKER_adj_close

data = data[['OMV.VI_adj_close', 'SIE.DE_adj_close', 'TSLA_adj_close', 'OMV.VI_market_cap', 'SIE.DE_market_cap','TSLA_market_cap', 'OMV.VI_total_debt', 'SIE.DE_total_debt', 'TSLA_total_debt']].dropna()    # Changing the order of columns and removing nan's




##################### The next part calculates V_a's and sigma_a #####################


tolerance = 0.0001                                            # Setting tolerance limit for the iterative procedure

num_iterations = 2                                            # Number of iterations

risk_free_rate = 0.02                                         # We assume this risk-free rate

time = 1                                                      # The time can be taken to be 1, as the lecturer, who responsible for this assignment said

sigma_a_list = []                                             # Initializing sigma_list


for i in range(len(tickers)):                                 # First (outer) loop for the three stocks

    V_a = 0                                                   # Initializing V_a value

    for j in range(num_iterations):                           # Second (middle) loop for the itearative procedure; the calculation in the thrid loop will be repeated here

      if j == 0:
        sigma_a = np.std(np.diff(np.log(data.iloc[:, i])), ddof = 1)             # The initial estimate for sigma_a is the standard deviation of the equity
      elif j > 0:
        sigma_a = np.std(np.diff(np.log((V_a_list))), ddof = 1)                  # The second and all further estimations for sigma_a will be calculated as the sample standard deviations of the estimated Va's

      V_a_list = []                                           # Initializing list for the Va's


      for k in range(len(data)):                              # Third (inner) loop for calculating a series of Va's

        d_1 = (np.log(V_a / (data.iloc[k, i + 6]+0.0000000001))+(risk_free_rate + 1/2 * sigma_a**2) * time)/(sigma_a * np.sqrt(time))               # Defining d_1 for the formula

        d_2 = d_1 - sigma_a * np.sqrt(time)                                                                                     # Defining d_2 for the formula

        def equation(V_a):
          return V_a * norm.ppf(d_1) - data.iloc[k, i + 6] * np.exp(-risk_free_rate * time) * norm.ppf(d_2) - data.iloc[k, i + 3]         # Defining equation to solve

        initial_guess = (1 + random.normalvariate(mu = 0, sigma = 1)) * data.iloc[k, i + 6]                                                                                          # Initial guess

        solution = fsolve(equation, initial_guess)                                                                              # Solution = a daily value for V_a

        V_a_list.append(solution)                                                                                               # Adding the V_a value to the list of V_a values


    sigma_a_list.append(sigma_a)

print(sigma_a_list)
